# Implementing Iteration

## Agenda

1. Review: Iteration
2. Details: *iterables*, *iterators*, `iter`, and `next`
3. Implementing iterators with classes
4. Implementing iterators with *generators* and `yield`

## 1. Iteration

*Iteration* simply refers to the process of accessing — one by one — the items stored in some container. The order of the items, and whether or not the iteration is comprehensive, depends on the container.

In Python, we typically perform iteration using the `for` loop.

In [ ]:
# e.g., iterating over a list
l = [2**x for x in range(10)]
for n in l:
    print(n)

In [ ]:
# e.g., iterating over the key-value pairs in a dictionary
d = {x:2**x for x in range(10)}
for k,v in d.items():
    print(k, '=>', v)

## 2. *iterables*, *iterators*, `iter`, and `next`

We can iterate over anything that is *iterable*. Intuitively, if something can be used as the source of items in a `for` loop, it is iterable.

But how does a `for` loop really work?

In [ ]:
l = [2**x for x in range(10)]

In [ ]:
it = iter(l)

In [ ]:
type(l), type(it)

In [ ]:
next(it)

In [ ]:
# putting it all together
it = iter(l)
while True:
    x = next(it)
    print(x)

In [ ]:
# iter & next are based on the __iter__ and __next__ special methods
it = l.__iter__()
while True:
    x = it.__next__()
    print(x)

## 3. Implementing iterators with classes

In [ ]:
class MyIterator:
    def __init__(self, max):
        self.max = max
        self.curr = 0
        
    # the following methods are required for iterator objects
    
    def __next__(self):
        pass
    
    def __iter__(self):
        pass

In [ ]:
it = MyIterator(10)

In [ ]:
next(it)

In [ ]:
it = MyIterator(10)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

In [ ]:
it = MyIterator(10)
for i in it:
    print(i)

An iterator is a *one time use object*! I.e., once we've used it to iterate over elements we cannot typically reset or "rewind" iteration. Iterable objects that can be traversed repeatedly return fresh iterators for each traversal.

In [ ]:
l = ['a', 'b', 'c', 'd', 'e']
for _ in range(3):
    for x in l:
        print(x, end=' ')

In [ ]:
l = ['a', 'b', 'c', 'd', 'e']
for _ in range(3):
    it = iter(l) # we obtain and "use up" an iterator each loop!
    while True:
        try:
            x = next(it)
            print(x, end=' ')
        except StopIteration:
            break

For a container type, we need to implement an `__iter__` method that returns an iterator.

In [108]:
import numpy as np

class ArrayListIterator:
    def __init__(self, array_list):
        self.array_list = array_list
        self.idx = 0


class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(len(self.data)*2, dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        return ArrayListIterator(self)

In [ ]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [ ]:
it = iter(l)

In [ ]:
type(it)

In [ ]:
next(it)

In [ ]:
for x in l:
    print(x)

## 4. Implementing iterators with generators

What's a "generator"?

In [ ]:
l = [2*x for x in range(10)]
g = (2*x for x in range(10))

In [ ]:
type(l), type(g)

In [ ]:
# try running this cell repeatedly
for x in l:
    print(x)

In [ ]:
# try running this cell repeatedly
for x in g:
    print(x)

In [ ]:
dir(g)

In [ ]:
%timeit -n 1000 [2*x for x in range(10_000)]

In [ ]:
%timeit -n 1000 (2*x for x in range(10_000))

In [ ]:
g = (2*x for x in range(10_000))

In [ ]:
g[100]

In [ ]:
g[:100]

In [ ]:
# try running this cell repeatedly
sum(g)

A *generator expression* syntactically resembles a list comprehension, and is similar in that it evaluates to an iterable sequence of values. However, a generator does not represent a fully fleshed out collection of values; instead, values are returned only as they are required through the iteration API (i.e., `next`) --- we refer to this as *lazy evaluation*. 

This makes a generator more efficient than a list (since we don't need to keep all values in the sequence around), but generators can't replace lists in all scenarios (e.g., when we need to jump around in the sequence or revisit values).

### Creating generator functions: `yield`

In [ ]:
def foo():
    yield

In [ ]:
foo()

In [ ]:
type(foo())

In [ ]:
def foo():
    print('hello!')
    yield
    print('goodbye!')

In [ ]:
foo()

In [ ]:
g = foo()

In [ ]:
next(g)

In [ ]:
def foo():
    yield 1
    yield 2
    yield 3

In [ ]:
g = foo()

In [ ]:
next(g)

In [ ]:
def countdown(n):
    for x in range(n, 0, -1):
        yield x
    yield 'Blast off!'

In [ ]:
for x in countdown(5):
    print(x)

In [ ]:
list(countdown(10))

A *generator function* is a function that contains one or more `yield` statements. When called, a generator function returns a generator object, which effectively allows us to incrementally execute the function using the iteration API. Each call to `next` on the generator will execute the function up to the next `yield` statement; if/when the function completes the generator will raise a `StopIteration` exception (just like an iterator).

### Generators as Data Structure Iterators

In [ ]:
class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(len(self.data)*2, dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        pass

In [ ]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [ ]:
for x in l:
    print(x)

In [ ]:
# use the now functioning iterator to implement __repr__
class ArrayList(ArrayList):
    def __repr__(self):
        pass

In [ ]:
l = ArrayList()
for x in range(10):
    l.append(2**x)
l